In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import spacy
import re

In [2]:
torch.cuda.is_available()

False

In [4]:
objaverse_dict = torch.load("../src/eval_data/objaverse_dict.pt")
ov_xyz = torch.load("../src/eval_data/ov_xyz.pt")
ov_rgb = torch.load("../src/eval_data/ov_rgb.pt")

ov_res_norm = torch.load("../src/eval_data/ov_confusion_norm.pt")

ov_confusion = torch.load("../src/eval_data/ov_confusion.pt")

In [5]:
arr = ["orange_(fruit)", "egg_yolk", "ping-pong_ball"]

# Define a function to modify each string
def modify_string(s):
    # Remove special characters and insert spaces
    s = re.sub(r'[^a-zA-Z0-9]+', ' ', s)
    return s

# Modify each string in the list
modified_arr = [modify_string(s) for s in arr]

# Print the modified list
print(modified_arr)

['orange fruit ', 'egg yolk', 'ping pong ball']


In [ ]:
# ['bottle', 'wine_bottle', 'beer_bottle', 'water_bottle', 'Tabasco_sauce']
disable_components = ["tagger", "parser", "attribute_ruler", "lemmatizer", "ner"]
nlp_lg = spacy.load("en_core_web_lg", disable=disable_components)

# objaverse_dict = torch.load("eval_data/objaverse_dict.pt")
ov_category2idx = objaverse_dict["category2idx"]

cats = list(ov_category2idx.keys())
sim_mat = np.zeros((len(cats), len(cats)))

doc_list = []
for i, cat in enumerate(cats):
    doc = nlp_lg(modify_string(cat))
    doc_list.append(doc)

for i, doc1 in enumerate(doc_list):
    for j, doc2 in enumerate(doc_list):
        sim_mat[i][j] = doc1.similarity(doc2)

/scratch/ipykernel_920561/3752770469.py:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim_mat[i][j] = doc1.similarity(doc2)


In [16]:
vodka_doc = nlp_lg("vodka bottle")

In [17]:
vod_arr = ["vodka", 'bottle', 'wine bottle', 'beer bottle', 'water bottle', 'Tabasco sauce']
doc_arr = []
for i, cat in enumerate(vod_arr):
    # doc = nlp_lg(modify_string(cat))
    doc_arr.append(nlp_lg(modify_string(cat)))

for d in doc_arr:
    print(vodka_doc.similarity(d))

0.8775750912088506
0.911566156245781
0.8583181620252078
0.9027107534849181
0.69818236796488
0.5011318153621519


In [9]:
# def top_confusion(topx=40, topy=5):
#     # modelnet_dict = torch.load("src/eval_data/modelnet_dict.pt")
#     # idx2category = modelnet_dict["idx2category"]
#     objaverse_dict = torch.load("eval_data/objaverse_dict.pt")
#     ov_idx2category = objaverse_dict["idx2category"]

#     # mn_res_norm = torch.load("src/eval_data/mn_confusion_norm.pt")
#     ov_res_norm = torch.load("eval_data/ov_confusion_norm.pt")


#     ov_res_norm = ov_res_norm.fill_diagonal_(0)
#     row_sum = torch.sum(ov_res_norm, dim=1)
#     # print("row_sum: ", row_sum.shape) # 1156 categories

#     sorted_idx = torch.argsort(row_sum, descending=True)
#     # print("row_sum: ", row_sum[sorted_idx[:200]] )
#     # print("sorted: ", sorted_idx)
#     sort_cat = [ov_idx2category[i.item()] for i in sorted_idx[:topx]]


#     for i in sorted_idx[:topx]:
#         topy_pred = torch.argsort(ov_res_norm[i], descending=True)[:topy]
#         topy_cat = [ov_idx2category[i.item()] for i in topy_pred]
#         print(ov_idx2category[i.item()], ": ", topy_cat)
#         # print(ov_idx2category[i.item()], ": ", ov_res_norm[i][topy_pred])


In [6]:
def top_confusion(objaverse_dict, ov_confusion, ov_res_norm, dim=1, topx=40, topy=5):
    # modelnet_dict = torch.load("src/eval_data/modelnet_dict.pt")
    # idx2category = modelnet_dict["idx2category"]
    # objaverse_dict = torch.load("eval_data/objaverse_dict.pt")
    ov_idx2category = objaverse_dict["idx2category"]

    # mn_res_norm = torch.load("src/eval_data/mn_confusion_norm.pt")
    # ov_res_norm = torch.load("eval_data/ov_confusion_norm.pt")

    ov_res_norm = ov_res_norm.fill_diagonal_(0)
    row_sum = torch.sum(ov_res_norm, dim=dim)
    # print("row_sum: ", row_sum.shape) # 1156 categories

    sorted_idx = torch.argsort(row_sum, descending=True)
    # print("row_sum: ", row_sum[sorted_idx[:200]] )
    # print("sorted: ", sorted_idx)
    sort_cat = [ov_idx2category[i.item()] for i in sorted_idx[:topx]]

    print("sorted: ", sorted_idx[:topx], '\n')
    top_xy_dict = {}

    for i in sorted_idx[:topx]:
        topy_cat=[]
        sim_list = []
        confusion = []
        top_xy_dict[i.item()] = {"cat_name":[], "count": [], "cats": []}
        topy_pred = torch.argsort(ov_res_norm[i], descending=True)[:topy]
        for cat_i in topy_pred:
            topy_cat.append(ov_idx2category[cat_i.item()])
            sim_list.append(sim_mat[i][cat_i])
            confusion.append(ov_confusion[i][cat_i].item())
        
        top_xy_dict[i.item()]["cats"] = topy_pred.tolist()
        top_xy_dict[i.item()]["cat_name"] = topy_cat
        top_xy_dict[i.item()]["count"] = confusion

        # topy_cat = [ov_idx2category[i.item()] for i in topy_pred]
        print(ov_idx2category[i.item()], ": ", topy_cat)
        # print(ov_idx2category[i.item()], ": ", sim_list)
        print(ov_idx2category[i.item()], ": ", confusion, '\n')
        # print(ov_idx2category[i.item()], ": ", ov_res_norm[i][topy_pred])
    
    # print("top_xy_dict: ", top_xy_dict)
    # np.save("src/eval_data/top_xy_dict.npy", top_xy_dict)


In [19]:
ov_category2idx = objaverse_dict["category2idx"] 
ov_category2idx["bridal_gown"]

142

In [8]:
top_confusion(objaverse_dict, ov_confusion, ov_res_norm)
# orange_(fruit), egg_yolk, ping-pong_ball

sorted:  tensor([1101,  142,  114,  169,  898,  426, 1025,  312,  438,  950,  758,  554,
         826, 1083,  118, 1037,  122,  612,  386,  902,  189,  568,  270,  903,
         464,  618,  278,  372,  724,  619,  658,  414,  667,  210,  229,  410,
         678,  483,  106, 1019]) 

vodka :  ['bottle', 'wine_bottle', 'beer_bottle', 'water_bottle', 'Tabasco_sauce']
vodka :  [17.0, 14.0, 6.0, 4.0, 3.0] 

bridal_gown :  ['dress', 'robe', 'skirt', 'dress_hat', 'swimsuit']
bridal_gown :  [17.0, 3.0, 2.0, 2.0, 2.0] 

boiled_egg :  ['egg', 'egg_yolk', 'person', 'orange_(fruit)', 'blimp']
boiled_egg :  [17.0, 2.0, 1.0, 1.0, 1.0] 

cabinet :  ['armoire', 'bookcase', 'dresser', 'fireplace', 'television_set']
cabinet :  [17.0, 9.0, 7.0, 4.0, 4.0] 

sherbert :  ['icecream', 'popsicle', 'plow_(farm_equipment)', 'pocket_watch', 'pliers']
sherbert :  [17.0, 2.0, 0.0, 0.0, 0.0] 

foal :  ['horse', 'pony', 'zebra', 'gazelle', 'dog']
foal :  [17.0, 2.0, 1.0, 1.0, 1.0] 

telephoto_lens :  ['camera_lens',

In [27]:
# top_confusion(objaverse_dict, ov_res_norm, dim=0)

In [7]:
def plot_pointcloud(objaverse_dict, ov_xyz, ov_rgb, list_instance, apply_color=True, marker_size=4, opacity=0.2):
    # objaverse_dict = torch.load("eval_data/objaverse_dict.pt")
    # ov_xyz = torch.load("eval_data/ov_xyz.pt")
    # ov_rgb = torch.load("eval_data/ov_rgb.pt")
    ov_idx2category = objaverse_dict["idx2category"]
    ov_category2idx = objaverse_dict["category2idx"]
    ov_labels_all = objaverse_dict["labels_all"]

    # idx = ov_category2idx[obj_name]
    # obj_instance = np.random.choice(torch.where(ov_labels_all == idx)[0])
    # print(f"obj_instance: {obj_instance}")

    for ins in list_instance:
        obj_xyz = ov_xyz[ins]
        obj_rgb = ov_rgb[ins]*255
        obj_rgb = obj_rgb.type(torch.int)
        df = pd.DataFrame(obj_xyz, columns=["x", "y", "z"])
        
        color_list = [f"rgb({r}, {g}, {b})" for [r, g, b] in obj_rgb]
        # color_list = ['rgb(227, 119, 194)']*7000 + ['rgb(119, 227, 152)']*3000
        
        fig = px.scatter_3d(df, x="x", y="y", z="z", opacity=0.1)
        if apply_color:
            fig = px.scatter_3d(df, x="x", y="y", z="z", color = color_list, color_discrete_map="identity", opacity=opacity)
        fig.update_traces(marker_size = marker_size)
        fig.show()
    

In [19]:
# def plot_multiple_same_obj(objaverse_dict, ov_xyz, ov_rgb, obj_name, num_ins):
#     ov_idx2category = objaverse_dict["idx2category"]
#     idx = ov_category2idx[obj_name]
#     ov_labels_all = objaverse_dict["labels_all"]
#     all_ins = torch.where(ov_labels_all == idx)[0]
#     list_ins = np.random.choice(all_ins, size = num_ins, replace=False)
#     plot_pointcloud(objaverse_dict, ov_xyz, ov_rgb, list_ins, apply_color=True)

In [8]:
# Given a list of object, it will plot random instances of those objects. Total num_ins plots
def plot_multiple_ins(objaverse_dict, ov_xyz, ov_rgb, obj_list, num_ins):
    ov_idx2category = objaverse_dict["idx2category"]
    ov_category2idx = objaverse_dict["category2idx"]
    idx = torch.tensor([ov_category2idx[o] for o in obj_list])
    # idx = ov_category2idx[obj_name]
    ov_labels_all = objaverse_dict["labels_all"]
    mask = torch.isin(ov_labels_all, idx)
    indices = torch.nonzero(mask).flatten()
    
    # all_ins = torch.where(ov_labels_all == idx)[0]
    list_ins = np.random.choice(indices, size = num_ins, replace=False)
    plot_pointcloud(objaverse_dict, ov_xyz, ov_rgb, list_ins, apply_color=True)
    objs = [ov_idx2category[o.item()] for o in ov_labels_all[list_ins]]
    return objs

In [9]:
# Where true is vodka but pred is water_bottle
def plot_multiple_confused_obj(objaverse_dict, ov_xyz, ov_rgb, true_obj, pred_obj_list, num_ins):
    ov_idx2category = objaverse_dict["idx2category"]
    ov_labels_all = objaverse_dict["labels_all"]
    
    idx = torch.tensor([ov_category2idx[o] for o in pred_obj_list])
    # idx = ov_category2idx[obj_name]
    ov_confusion
    
    all_ins = torch.where(ov_labels_all == idx)[0]
    list_ins = np.random.choice(all_ins, size = num_ins, replace=False)
    plot_pointcloud(objaverse_dict, ov_xyz, ov_rgb, list_ins, apply_color=True)

In [2]:
# objs = plot_multiple_ins(objaverse_dict, ov_xyz, ov_rgb, ['sherbert', 'icecream', 'popsicle', 'plow_(farm_equipment)', 'pocket_watch', 'pliers'], 4)
objs = plot_multiple_ins(objaverse_dict, ov_xyz, ov_rgb, ['shopping_bag'], 1)

In [31]:
objs

['pickle', 'cucumber', 'pickle']

In [33]:
objs

['fighter_jet',
 'fighter_jet',
 'fighter_jet',
 'fighter_jet',
 'fighter_jet',
 'fighter_jet']

In [15]:
sum(ov_confusion[1101])

tensor(51.)

In [16]:
ov_confusion[1101][1101]

tensor(3.)

In [19]:
ov_copy = ov_confusion.clone()

In [21]:
ov_copy = ov_copy.fill_diagonal_(0)

In [24]:
ov_confusion

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  2.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  9.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0., 11.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  4.]])

In [22]:
ov_copy

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [27]:
sum(ov_copy[1101, :])

tensor(48.)

In [23]:
ov_category2idx["vodka"]

1101